In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Load the dataset

df = pd.read_csv('drug_consumption.data')
for i,column in enumerate(df.columns):
    print("### "+str(i+1)+". "+ column)


### 1. ID
### 2. AGE
### 3. GENDER
### 4. EDUCATION
### 5. COUNTRY
### 6. ETHNICITY
### 7. NSCORE
### 8. ESCORE
### 9. OSCORE
### 10. ASCORE
### 11. CSCORE
### 12. IMPULSIVE
### 13. SS
### 14. ALCOHOL
### 15. AMPHET
### 16. AMYL
### 17. BENZOS
### 18. CAFF
### 19. CANABIS
### 20. CHOC
### 21. COKE
### 22. CRACK
### 23. ECSTASY
### 24. HEROIN
### 25. KETAMINE
### 26. LEGALH
### 27. LSD
### 28. METH
### 29. MUSHROOMS
### 30. NICOTINE
### 31. SEMER
### 32. VSA
